# Introduction to Self-Driving Car Decision Making!


In [ ]:
#@title Run this for setup! (may take several minutes to finish all imports) Ignore any errors!
# quiet dependency warnings
import warnings
warnings.filterwarnings("ignore")

!pip install -q pyyaml==5.1 &> /dev/null
!pip install torch &> /dev/null
!pip install 'git+https://github.com/facebookresearch/detectron2.git' &> /dev/null #takes up to 5 minutes

In [ ]:
#@title Then run this for more setup!
# check pytorch installation:
import torch, torchvision
#print(torch.__version__, torch.cuda.is_available())

# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
from skimage.transform import resize
from PIL import Image
from pandas import DataFrame
from google.colab import data_table
from tqdm import tqdm
from ipywidgets import interact, IntSlider
import time
import pandas

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

!wget -q --show-progress "https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Pioneers/Self%20Driving%20Cars/gtFine.zip"
!wget -q --show-progress "https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Pioneers/Self%20Driving%20Cars/leftImg8bit.zip"
!mkdir /content/gtFine/
!mkdir /content/leftImg8bit/
!mkdir /content/gtFine/train
!mkdir /content/leftImg8bit/train

!unzip -qo /content/gtFine.zip -d /content/gtFine/train && mv /content/gtFine/train/gtFine/* /content/gtFine/train && rm -rf /content/gtFine/train/gtFine /content/gtFine/train/__MACOSX
!unzip -qo /content/leftImg8bit.zip -d /content/leftImg8bit/train && mv /content/leftImg8bit/train/leftImg8bit/* /content/leftImg8bit/train && rm -rf /content/leftImg8bit/train/leftImg8bit /content/leftImg8bit/train/__MACOSX

X = []
y = []

for path in tqdm(sorted(os.listdir("/content/leftImg8bit/train"))):
  if path.endswith("png"):
    path = os.path.join('/content/leftImg8bit/train', path)
    img = cv2.imread(path,cv2.IMREAD_COLOR)
    img = cv2.resize(img,(512, 256))
    X.append(img)

for path in tqdm(sorted(os.listdir("/content/gtFine/train"))):
  if path.endswith("color.png"):
    path = os.path.join('/content/gtFine/train', path)
    img = cv2.imread(path,cv2.IMREAD_COLOR)
    img = cv2.resize(img,(512, 256))
    y.append(img)

X, input_data= np.array(X), np.array(X)
y, output_data = np.array(y), np.array(y)

class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'handbag', 'tie', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'keyboard', 'oven', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
class_list = DataFrame(class_list, columns=['Class name'])

output_label = ['GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'STOP!', 'GO!', 'GO!', 'STOP!', 'STOP!', 'GO!', 'STOP!', 'GO!', 'GO!', 'STOP!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'STOP!', 'GO!', 'GO!', 'GO!', 'STOP!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'STOP!', 'STOP!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'STOP!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'STOP!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'STOP!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'STOP!', 'GO!', 'GO!', 'GO!', 'GO!', 'STOP!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'STOP!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'STOP!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'STOP!', 'GO!', 'GO!', 'GO!', 'GO!', 'STOP!', 'STOP!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'STOP!', 'STOP!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'STOP!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'STOP!', 'GO!', 'GO!', 'STOP!', 'GO!', 'GO!', 'STOP!', 'STOP!', 'STOP!', 'STOP!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'STOP!', 'GO!', 'GO!', 'STOP!', 'GO!', 'GO!', 'STOP!', 'GO!', 'GO!', 'GO!', 'GO!', 'GO!', 'STOP!', 'STOP!', 'GO!', 'GO!', 'GO!', 'STOP!', 'STOP!', 'GO!']
output_label = DataFrame(output_label, columns=['Decision'])

def show_one_image(index):
  cv2_imshow(input_data[index])

class ObjectDetectionModel():
  def __init__(self):
    self.cfg = None
    self.predictor = None
  def fit(self, X, y, pretrained=True):
    self.cfg = get_cfg()
    # add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
    self.cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_50_3x.yaml"))
    self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
    # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
    self.cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_50_3x.yaml")
    self.predictor = DefaultPredictor(self.cfg)
  def predict(self, image):
    outputs = self.predictor(image)
    return outputs
  def show_image(self, outputs, image):
    cv2_imshow(image)
    v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])


class ImageSegmentationModel():
  def __init__(self):
    self.cfg = None
    self.predictor = None
  def fit(self, X, y, pretrained=True):
    self.cfg = get_cfg()
    # add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
    self.cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_50_3x.yaml"))
    # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
    self.cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_50_3x.yaml")
    self.predictor = DefaultPredictor(self.cfg)
  def predict(self, image):
    panoptic_seg, segments_info = self.predictor(image)["panoptic_seg"]
    return (panoptic_seg, segments_info)
  def show_image(self, outputs, image):
    panoptic_seg, segments_info = outputs
    v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
    cv2_imshow(out.get_image()[:, :, ::-1])

# Exploring the Cityscapes Dataset

In [ ]:
for i in range(5):
  show_one_image(i)

**What sort of data are we working with? What kind of objects are captured in the data?**

In [ ]:
#@title Select a target image and explore its segmentation map!
def show_segmented_image(index):
  cv2_imshow(input_data[index])
  cv2_imshow(output_data[index])

interact(show_segmented_image, index=IntSlider(min=0, max=199, step=1, value=0))



### Exercise: Getting our data shape
Please print out the **number of images**, along with the **height and width** of each image. Let's print out the shape of our output data too!

In [ ]:
input_num_images, input_height, input_width, _ = input_data.shape
output_num_images, output_height, output_width, _ = output_data.shape

### YOUR CODE HERE ###
print("Number of Input Images in Dataset:", ___)
print("Height of Input Image(pixels):", ___)
print("Width of Input Image(pixels):", ___)
print()
print("Number of Output Images in Dataset:", ___)
print("Height of Output Image(pixels):", ___)
print("Width of Output Image(pixels):", ___)

**Discussion Question:** Why are the values for the input and output data shape the same?

# Creating an Object Detection Model

In [ ]:
model = ObjectDetectionModel()

### Step 2: Train your model!

In [ ]:
### YOUR CODE BEGINS HERE ###

### YOUR CODE ENDS HERE ###

### Step 3: Make Predictions using your Model!

In [ ]:
selected_index = 124 # Choose any image in our dataset
show_one_image(selected_index)

In [ ]:
image = input_data[selected_index]
### YOUR CODE BEGINS HERE ###

### YOUR CODE ENDS HERE ###

### Step 4: Visualize our predictions!

In [ ]:
### YOUR CODE BEGINS HERE ###

### YOUR CODE ENDS HERE ###

In [ ]:
#@title Try your model on any image in our dataset
def run_model(selected_index):
  im = input_data[selected_index]
  outputs = model.predict(im)

  model.show_image(outputs, im)

interact(run_model, selected_index=IntSlider(min=0, max=199, step=1, value=0))

In [ ]:
### YOUR CODE BEGINS HERE ###

### YOUR CODE ENDS HERE ###
model.show_image(outputs, image)

In [ ]:
#@title Run this to initialize output variables!
output_instances = outputs["instances"]
pred_classes = output_instances.pred_classes.tolist()
scores = output_instances.scores.tolist()

**Exercise:** Print out the output variables.

In [ ]:
print(scores)

In [ ]:
from google.colab import data_table

data_table.DataTable(class_list)

In [ ]:
output_classes = ['class name 1', 'class name 2'] # Replace 'class name 1', 'class name 2', etc. with the class names from above

### Advanced Exercise: Write a for loop

In [ ]:
output_classes = []
### YOUR CODE BEGINS HERE ###

### YOUR CODE ENDS HERE ###
print(output_classes)

For now, we'll say that the car should stop given the following conditions:


1.   If a bike is detected in the image, stop the car.
2.   If a person is detected in the image, stop the car.
3.   Otherwise, keep moving the car forward.

Now let's code this up! You could imagine the function predict_movement() looking a little like this:

```
def predict_movement(pred_classes):
  if(_____ in pred_classes):
    return "STOP!"
  else:
    return "GO!"
```

**Discussion Question:** What values should we be putting into the blanks?

Let's do some investigating and figure out which indexes correspond to a bike or a person so we can tell our car what to do!


In [ ]:
#@title Find out which class indexes are for bike and person
def show_object_index(class_index):
  return class_list['Class name'][class_index]

interact(show_object_index, class_index=IntSlider(min=0, max=70, step=1, value=0))


In [ ]:
def predict_movement(pred_classes):
  if ______ in pred_classes:
    return "STOP!"
  #TODO: Add condition for bike and other objects here
  else:
    return "GO!"

In [ ]:
#@title Run our decision-making model on images of your choice
def run_decision_making_model(selected_index):
  im = input_data[selected_index]
  outputs = model.predict(im)

  model.show_image(outputs, im)
  output_instances = outputs["instances"]
  pred_classes = output_instances.pred_classes.tolist()

  print("Final decision:", predict_movement(pred_classes))
interact(run_decision_making_model, selected_index=IntSlider(value=0, min=0, max=199, step=1))

In [ ]:
#@title Get Decision Model Accuracy! (may take a minute to run)
predictions = []
for image in tqdm(input_data):
  outputs = model.predict(image)
  output_instances = outputs["instances"]
  pred_classes = output_instances.pred_classes.tolist()

  predictions.append(predict_movement(pred_classes))

output_label['Predicted Decision'] = predictions

print("\nAccuracy:", len([0 for i, pred in enumerate(predictions) if output_label['Decision'][i] == pred])/len(output_data))
data_table.DataTable(output_label)

In [ ]:
#@title Set up our crop boundary!

left_crop_final = IntSlider(min=0, max=511, step=1, value=0)
right_crop_final = IntSlider(min=0, max=511, step=1, value=511)
bottom_crop_final = IntSlider(min=0, max=255, step=1, value=255)
top_crop_final = IntSlider(min=0, max=255, step=1, value=0)

def show_cropped_image(selected_index, left_crop, right_crop, bottom_crop, top_crop):
  im = input_data[selected_index]
  crop_img = im[top_crop:bottom_crop, left_crop:right_crop]
  outputs = model.predict(crop_img)
  model.show_image(outputs, crop_img)

interact(show_cropped_image,
         selected_index=IntSlider(min=0, max=199, step=1, value=0),
         left_crop=left_crop_final,
         right_crop=right_crop_final,
         bottom_crop=bottom_crop_final,
         top_crop=top_crop_final
         )


In [ ]:
#@title Get Cropped Model Accuracy! (may take a minute to run)
predictions = []
for image in tqdm(input_data):
  crop_img = image[top_crop_final.value:bottom_crop_final.value, left_crop_final.value:right_crop_final.value]
  outputs = model.predict(crop_img)
  output_instances = outputs["instances"]
  pred_classes = output_instances.pred_classes.tolist()

  predictions.append(predict_movement(pred_classes))

output_label['Predicted Decision'] = predictions
print("\nAccuracy:", len([0 for i, pred in enumerate(predictions) if output_label['Decision'][i] == pred])/len(output_data))
output_label

In [ ]:
#@title Run this to view misclassified images!
decision = output_label['Decision']
pred_decision = output_label['Predicted Decision']
misclassified = decision[pred_decision != decision]
for i in range(10):
  cv2_imshow(input_data[misclassified.index[i]])
  print("Our Prediction: ", output_label['Predicted Decision'][misclassified.index[i]])
  print("Correct Prediction: ", output_label['Decision'][misclassified.index[i]])


In [ ]:
seg_model = ImageSegmentationModel()

In [ ]:
### YOUR CODE HERE ###

In [ ]:
segment_index = 0 # Choose any image from our input data

image = input_data[segment_index]
### YOUR CODE BEGINS HERE ###

### YOUR CODE ENDS HERE ###

In [ ]:
seg_model.show_image(outputs, image)

In [ ]:
#@title Try your model on any image from our data!
def run_model(selected_index):
  im = input_data[selected_index]
  outputs = seg_model.predict(im)

  seg_model.show_image(outputs, im)

interact(run_model, selected_index=IntSlider(min=0, max=199, step=1, value=0))

**Discussion Question**: What additional information does our Image Segmentation model provide compared to our Object Detection Model?